In [3]:
import argparse
import easydict

import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchmetrics.functional.classification import accuracy

from src.engines import pretrain_baseline
from src.models import BaselineNet
from src.utils import save_checkpoint, save_pretrained_embeddingnet

# Jupyter 환경
args = easydict.EasyDict({
    "title" : "transfer",
    "device" : "cuda",
    "root" : "data/omniglot/meta-train",
    "num_workers" : 2,
    "num_classes" : 1432,
    "batch_size" : 16,
    "epochs" : 100,
    "lr" : 0.001,
    "checkpoints" : 'checkpoints',
    "pretrain" : True
})

def main():
    # Build dataset : train만 진행
    train_transform = T.Compose([
        T.Grayscale(),
        T.RandomCrop((32, 32), padding=4),
        T.ToTensor(),
    ])
    train_data = ImageFolder("data/omniglot/meta-train", transform=train_transform)
    train_loader = DataLoader(train_data, 16, shuffle=True, num_workers=2, drop_last=True)

    print(train_data)
    print(train_loader)
    for se, sl in train_data:
        print("[test_dataset] support_example shape : ", se.shape)
        print("[test_dataset] support_label value : ", sl)
        break
    for se, sl in train_loader:
        print("[test_loader] support_example shape : ", se.shape)
        print("[test_loader] support_label shape : ", sl.shape)
        print("[test_loader] support_label value : ", sl)
        break
        
    # Build model
    model = BaselineNet(args.num_classes, pretrain=args.pretrain)

    model = model.to(args.device)

    # Build optimizer 
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs * len(train_loader))
    loss_fn = nn.CrossEntropyLoss()
    metric_fn = accuracy

    # Main loop : train만 진행
    for epoch in range(1):
        train_summary = pretrain_baseline(train_loader, model, optimizer, scheduler, loss_fn, metric_fn, args.device)
        
        print(f'Epoch: {epoch + 1}, Train Accuracy: {train_summary["metric"]:.4f}')
        save_checkpoint(args.checkpoints, "debug_transfer", model, optimizer, epoch + 1)
    
    save_pretrained_embeddingnet(args.checkpoints, "debug_transfer", model.features) # model이 아닌, model.features를 인수로 사용 ♣
    
    model_dict = model.state_dict() # 현재 신경망 상태 로드
    print("[model_dict] : ", model_dict.keys())
    checkpoint_path = f'{args.checkpoints}/{debug_transfer}_embedding.pth'
    pretrained_dict = torch.load(checkpoint_path)  # pretrained 상태 로드
    print("[pretrained_dict] : ", pretrained_dict.keys())

if __name__=="__main__":
    main()

Dataset ImageFolder
    Number of datapoints: 28640
    Root location: data/omniglot/meta-train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               RandomCrop(size=(32, 32), padding=4)
               ToTensor()
           )
[test_dataset] support_example shape :  torch.Size([1, 32, 32])
[test_dataset] support_label value :  0
[test_loader] support_example shape :  torch.Size([16, 1, 32, 32])
[test_loader] support_label shape :  torch.Size([16])
[test_loader] support_label value :  tensor([1246, 1243,  970,  538,   76,  608, 1299,  423, 1234,  373,   29,  234,
         166,   88,  302,  464])
Epoch: 1, Train Accuracy: 0.0012
[model_dict] :  odict_keys(['features.layers.0.conv.weight', 'features.layers.0.conv.bias', 'features.layers.0.norm.weight', 'features.layers.0.norm.bias', 'features.layers.0.norm.running_mean', 'features.layers.0.norm.running_var', 'features.layers.0.norm.num_batches_tracked', 'features.layers.1.conv.weight', 'fe

NameError: name 'debug_transfer' is not defined